Import everything we need first

In [ ]:
import time
START = time.time()

import warnings
warnings.simplefilter("ignore")

import itertools
import sys
import gc
import json
import random
import os
import warnings
from copy import copy, deepcopy

import numpy as np
import pandas as pd

from scipy.optimize import minimize
from lightgbm import LGBMRegressor
from bayes_opt import BayesianOptimization

from math import sqrt

Reading train data.

Keep only installations with assessment attempts - we can't calcullate train targets for other anyway.

Map categorical columns to integer.

In [ ]:
train = pd.read_csv("/kaggle/input/data-science-bowl-2019/train.csv")

trainable = train[(train.type == 'Assessment') & (
    (train.title == 'Bird Measurer (Assessment)') & (train.event_code == 4110) |
    (train.title != 'Bird Measurer (Assessment)') & (train.event_code == 4100)
)].installation_id.drop_duplicates()
train = train[train.installation_id.isin(trainable)]

train.sort_values(["installation_id", "timestamp", "event_count"], inplace=True)

alltypes = list(train.type.drop_duplicates())
alltitles = list(train.title.drop_duplicates())
allevents = list(train.event_code.drop_duplicates())
allworlds = list(train.world.drop_duplicates())

CATEGORICAL = [
    'previous_title',
    'previous_type',
    'previous_world',
    'current_title',
    'current_world',
    'assessment_world',
    'assessment_title',
]

CATMAPS = {} # will be filled in generate_features

Big function to generate features from train or test data.

Each assessment featured for train mode, only last - for test

It is A LOT of data in source train/test files, only some aggregates kept as features - this is probably main point to improve results

In [ ]:
def generate_features(df, only_test=False):
    
    global CATMAPS
    
    df["timestamp"] = df.timestamp.apply(lambda x : pd.Timestamp(x))

    total = df.installation_id.nunique()

    X = []
    
    start = time.time()
    cnt = 0
    
    for i, d in df.groupby("installation_id"):
        d.sort_values(["timestamp", "event_count"], inplace = True)

        d["is_attempt"] = ((d.type == 'Assessment') & (
                           ((d.title == 'Bird Measurer (Assessment)') & (d.event_code == 4110)) |
                           ((d.title != 'Bird Measurer (Assessment)') & (d.event_code == 4100))
                          ))
        attempt_sessions = list(d[d.is_attempt].game_session.drop_duplicates())

        d["for_train"] = (d.game_session.isin(attempt_sessions)) & (~d.game_session.duplicated(keep='first'))
        
        attempt_sessions = set(attempt_sessions)
        
        for_test = (d.type == "Assessment") & (~d.game_session.duplicated(keep='first'))
        for_test = for_test[for_test].index[-1]
        d["for_test"] = False
        d["for_test"][for_test] = True
        
        first_x = {
            "events_count": 0,
            "sessions_count": 0,
            "sessions_duration": 0,
            "age": 0.0,
            "nonass_attempts": 0,
            "nonass_successes": 0,
            
            # Categorical
            "current_world": "",
            "current_title": "",
            "previous_world": "",
            "previous_title": "",
            "previous_type": "",
        }
        for x in alltypes:
            first_x["count_type_%s" % x] = 0
            first_x["duration_type_%s" % x] = 0
        for x in alltitles:
            first_x["count_title_%s" % x] = 0
            first_x["duration_title_%s" % x] = 0
            
            first_x["last_noass_round_%s" % x] = 0
            first_x["last_noass_level_%s" % x] = 0
            
            first_x["noass_attempts_%s" % x] = 0
            first_x["noass_successes_%s" % x] = 0
            
        for x in allevents:
            first_x["count_event_%s" % x] = 0
            for y in alltitles:
                first_x["count_titlevent_%s_%s" % (y, x)] = 0
        for x in allworlds:
            first_x["count_world_%s" % x] = 0
            first_x["duration_world_%s" % x] = 0

        second_x = {
            "assessment_order" : 0,
            "total_attempts": 0,
            "total_successes": 0,
            "last_attempts" : 0,
            "last_successes" : 0,
            
            # Categorical
            "assessment_world": "",
            "assessment_title": "",
        }
        for x in alltitles:
            if 'Assessment' in x:
                second_x["count_assessment_%s" % x] = 0
                second_x["attempts_assessment_%s" % x] = 0
                second_x["successes_assessment_%s" % x] = 0
                    
        for x in allworlds:
            second_x["world_assessment_count_%s" % x] = 0
            second_x["world_assessment_attempts_%s" % x] = 0
            second_x["world_assessment_successes_%s" % x] = 0

        session = ""
        session_type = ""
        session_title = ""
        session_world = ""
        session_start = 0
        session_duration = 0
        session_events = 0
        
        assessment = ""
        assessment_attempts = 0
        assessment_success = 0
        assessment_start = 0
        
        prev_details = {"session": ''}
        delta_X = []
        delta_Y = []
        delta_T = []
        
        
        
        session_intervals = []
        
        for j, r in d.iterrows():
            
            details = json.loads(r["event_data"])
            details["session"] = r["game_session"]

            if details["session"] == prev_details["session"]:
                if "coordinates" in details and "coordinates" in prev_details:
                    delta_X.append(abs(details["coordinates"]["x"] - prev_details["coordinates"]["x"]))
                    delta_Y.append(abs(details["coordinates"]["y"] - prev_details["coordinates"]["y"]))
                    delta_T.append(details["game_time"] - prev_details["game_time"])
            prev_details = details
            
            if "correct" in details:
                if r["type"] != "Assessment":

                    first_x["nonass_attempts"] += 1                    
                    first_x["noass_attempts_%s" % r["title"]] += 1

                    if details["correct"]:
                        first_x["nonass_successes"] += 1
                        first_x["noass_successes_%s" % r["title"]] = 0
                        
                        if "level" in details:
                            first_x["last_noass_level_%s" % r["title"]] = details["level"]
                        
                        if "round" in details:
                            first_x["last_noass_round_%s" % r["title"]] = details["round"]
    
            
            first_x["events_count"] += 1
            first_x["count_event_%s" % r["event_code"]] += 1
            first_x["count_titlevent_%s_%s" % (r["title"], r["event_code"])] += 1
            
            first_x["weekday"] = r["timestamp"].weekday()
            first_x["monthpart"] = r["timestamp"].day / float(r["timestamp"].daysinmonth)
            first_x["daytime"] = (r["timestamp"].value / 10**9 % (3600*24)) / 3600. / 24.
                
            if r["game_session"] == session:
                session_duration = r["timestamp"].value / 10**6 - session_start
                session_events += 1
            
            else:
                first_x["sessions_count"] += 1
                
                first_x["sessions_duration"] += session_duration
                first_x["last_session_duration"] = session_duration
                first_x["age"] +=  session_duration
                session_intervals.append(session_duration)
                first_x["last_session_events"] = session_events
                first_x["since_last_session"] = r["timestamp"].value / 10**6 - session_start - session_duration
            
                if session != "":
                    first_x["count_type_%s" % session_type] += 1
                    first_x["duration_type_%s" % session_type] += session_duration

                    first_x["count_title_%s" % session_title] += 1
                    first_x["duration_title_%s" % session_title] += session_duration
                    
                    first_x["count_world_%s" % session_world] += 1
                    first_x["duration_world_%s" % session_world] += session_duration
                    
                second_x["last_attempts"] = assessment_attempts
                second_x["last_successes"] = assessment_success
               
                session = r["game_session"]
                session_start = r["timestamp"].value / 10**6
                session_duration = 0
                session_events = 0
                session_type = r["type"]
                session_title = r["title"]
                session_world = r["world"]

            if r["for_train"] or (r["for_test"] and only_test):
                second_x["assessment_order"] += 1
                second_x["count_assessment_%s" % r["title"]] += 1
                second_x["world_assessment_count_%s" % r["world"]] += 1
                
                first_x["current_world"] = r["world"]
                first_x["current_title"] = r["title"]
                
                if not only_test or r["for_test"]:
                    
                    moves = pd.DataFrame({
                        'x': delta_X,
                        'y': delta_Y,
                        't': delta_T
                    })
                    moves["dist"] = moves.x * moves.x + moves.y * moves.y
                    moves["dist"] = moves.dist.apply(sqrt)
                    moves = moves[moves.t > 0]
                    
                    first_x["moves_count"] = moves.shape[0]
                    first_x["avg_move_duration"] = moves.t.mean()
                    first_x["avg_move_distanace"] = moves.dist.mean()
                    first_x["avg_total_speed"] = moves.dist.sum() / moves.t.sum()
                    first_x["avg_move_speed"] = (moves.dist / moves.t).mean()
                    first_x["avg_x_speed"] = (moves.x / moves.t).mean()
                    first_x["avg_y_speed"] = (moves.y / moves.t).mean()
                    
                    first_x["session_interval_mean"] = np.mean(session_intervals)
                    first_x["session_interval_std"] = np.std(session_intervals)
                    
                    xx1 = copy(first_x)
                    xx1["game_session"] = r["game_session"]
                    xx1["installation_id"] = r["installation_id"]

                    xx2 = copy(second_x)
                    xx1.update(xx2)
                    X.append(xx1)
            
            else:
                first_x["previous_world"] = r["world"]
                first_x["previous_title"] = r["title"]
                first_x["previous_type"] = r["type"]
                
            if r["is_attempt"]:
                second_x["assessment_world"] = r["world"]
                second_x["assessment_title"] = r["title"]
                
                second_x["total_attempts"] += 1
                second_x["attempts_assessment_%s" % r["title"]] += 1
                second_x["world_assessment_attempts_%s" % r["world"]] += 1
                
                if r["game_session"] ==  assessment:
                    assessment_attempts += 1
                else:
                    assessment = r["game_session"]
                    assessment_attempts = 1
                    assessment_success = 0
                    assessment_start = session_start
                    
                if json.loads(r["event_data"])["correct"]:
                    second_x["total_successes"] += 1
                    second_x["successes_assessment_%s" % r["title"]] += 1
                    second_x["world_assessment_successes_%s" % r["world"]] += 1
                    
                    if r["game_session"] ==  assessment:
                        assessment_success += 1
        cnt += 1
        if cnt % 100 == 0:
            print(float(cnt) / total, time.time() - start)
            sys.stdout.flush()
            
    X = pd.DataFrame(X)
    
    for col in list(X.columns):
        if 'count' in col:
            if col.replace("count", "duration") in list(X.columns):
                X[col.replace("count", "mean")] = X[col.replace("count", "duration")] / X[col]
                
    for col in list(X.columns):
        if 'count' in col:
            X[col.replace("count", "freq")] = X[col] / X["age"]
                
    def to_group(accuracy):
        if accuracy < 0.005:
            return 0
        elif 0.995 < accuracy < 1.005:
            return 3
        elif 0.4995 < accuracy < 0.5005:
            return 2
        else:
            return 1
        
    for col in list(X.columns):
        if 'attempts' in col:
            if col.replace("attempts", "successes") in list(X.columns):
                X[col.replace("attempts", "accuracy")] = X[col.replace("attempts", "successes")] / X[col]
                X[col.replace("attempts", "group")] = X[col.replace("attempts", "accuracy")].apply(to_group)
    
    print(CATMAPS.keys())
    if not only_test:
        for col in CATEGORICAL:
            uniques = list(X[col].drop_duplicates())
            uniques.sort()
            print(col)
            print(str(len(uniques)))
            CATMAPS[col] = {x : i for i, x in enumerate(uniques)}
            X[col] = X[col].apply(lambda x : CATMAPS[col][x])
    else:
        for col in CATEGORICAL:
            uniques = list(X[col].drop_duplicates())
            print(col)
            print(str(len(uniques)))
            X[col] = X[col].apply(lambda x : CATMAPS[col][x])
    
    X.columns = [x.replace(" ", "_").replace("'", "").replace("(", "").replace(")", "").replace(",", "") for x in X.columns] 
    
    X = X[sorted(X.columns)]
    
    return X

Generate features for train and test.

Delete large source log data.

In [ ]:
X = generate_features(train, only_test=False)

del train
gc.collect()

X.shape

In [ ]:
test = pd.read_csv("/kaggle/input/data-science-bowl-2019/test.csv")
test.sort_values(["installation_id", "timestamp", "event_count"], inplace=True)

Xtest = generate_features(test, only_test=True)

del test
gc.collect()

Xtest.shape

Drop constant features.

In [ ]:
todrop = []

for i, col in enumerate(X.columns):
    if X[col].nunique() < 2:
        todrop.append(col)
    elif Xtest[col].nunique() < 2:
        todrop.append(col)

print("Totally constant features:", len(todrop))

X.drop(todrop, axis=1, inplace = True)
Xtest.drop(todrop, axis=1, inplace = True)

X.shape

In [ ]:
X.to_csv("X.tsv", sep = '\t', index=False)
Xtest.to_csv("Xtest.tsv", sep = '\t', index=False)

Read target labels, move ids and target values to separate data frame y

In [ ]:
y = pd.read_csv("/kaggle/input/data-science-bowl-2019/train_labels.csv")

assert(X.shape[0] == y.shape[0])
X = pd.merge(X, y[['game_session', "installation_id", "accuracy_group"]])
assert(X.shape[0] == y.shape[0])

X.sort_values("installation_id", inplace=True)

y = X[["installation_id", "game_session", "accuracy_group"]]
X.drop(["accuracy_group", "installation_id", "game_session"], axis=1, inplace=True)

X.shape

Prepare submission table

In [ ]:
submission = pd.DataFrame({
    "installation_id": Xtest["installation_id"],
    "total_rank": 0
})

Xtest = Xtest[list(X.columns)]
Xtest.shape

4-fold CV

1) All assessments of one installation_id in one fold

2) Each fold have same number of installations

3) Each fold is balanced in terms of installations with one or many assessments

As a result, we should have roughly equal number of points in folds

In [ ]:
y["fold"] = -1

installations = sorted(list(y.installation_id.value_counts().items()), key = lambda x : -x[1])

for i, item in enumerate(installations):
    y.loc[y.installation_id == item[0], "fold"] = i % 4
    
y.fold.value_counts()

Simple OOF-score on CV isn't a good indication of model quality.

In our train set we have a lot of points for installations with many assessments - in test set only one point per installation.

Points with big assessment history are easier for model to evaluate => OOF score will be much better then actual test score.

To remove this bias we will "truncate" our train set to one point per installation for score evaluation.

In [ ]:
def truncate_df(df):

    indices = []

    for i, d in df.groupby("installation_id"):
        indices.append(random.choice(list(d.index)))

    return df.loc[indices]

Let's calculate truncated OOF target distribution for each of our CV folds

In [ ]:
cv_truncated_target_ditribution = {}

for fold in range(4):
    cv_truncated_target_ditribution[fold] = truncate_df(y[y.fold != fold]).accuracy_group.value_counts()
    for i in range(19):
        cv_truncated_target_ditribution[fold] += truncate_df(y[y.fold != fold]).accuracy_group.value_counts()
    cv_truncated_target_ditribution[fold] /= 20.0 * y[y.fold != fold].installation_id.nunique()
    
assert(3.99 < sum(cv_truncated_target_ditribution.values()).sum() < 4.01)
cv_truncated_target_ditribution

And truncated target distribution for all dataset.

Interesting, that it is really close to 50%-25%-12.5%-12.5%

In [ ]:
hist = sum(cv_truncated_target_ditribution.values()) / 4.0
hist

QWK score function - copy/pasted from one of competition's kernels

In [ ]:
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e

Main model-training function.

We train separte model for each fold with early stopping by RMSE metric.

And use multiple truncation of test set to get QWK distribution.

We don't use any thresholds to translate our continious predictions to classes.

Instead, we use predictions fo ranking and distribute labels similar to train distribution.

This can be punished heavily by test set with only one class for example, but we assume that test and train data randomly sampled from same distribution.

In [ ]:
ALL_CHECKED_CONFIGS = []

def check_cv_config(num_leaves, learning_rate):
    
    params = {
        'n_estimators': 10000,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'subsample': 0.75,
        'subsample_freq': 1,
        'learning_rate': learning_rate,
        'feature_fraction': 0.9,
        'lambda_l1': 1,  
        'lambda_l2': 1,
        'num_leaves': int(num_leaves),
        'random_state': 42,
        'n_jobs': 4,
        'importance_type' : 'gain',
    }
    
    FOLDED_MODELS = []
    FOLDED_SCORES = []
    
    for fold in range(4):
        
        if time.time() - START > 3600 * 8:
            raise Exception("OUT OF TIME!")

        clf = LGBMRegressor(**params)
        
        Xtr = X[y.fold != fold]
        ytr = y[y.fold != fold]
        Xte = X[y.fold == fold]
        yte = y[y.fold == fold]
        
        clf.fit(Xtr,
                ytr.accuracy_group,
                categorical_feature = [x for x in X.columns if x in CATEGORICAL],
                early_stopping_rounds = 100,
                eval_set = [(Xte, yte.accuracy_group)],
                verbose=False)
        
        pred = clf.predict(Xte)
        
        oof = yte[["installation_id", "accuracy_group"]]
        yte["prediction"] = pred
        
        scores = []
        
        for idx in range(10):
            
            df = truncate_df(yte)
                  
            df.sort_values("prediction", inplace = True)
            df["predicted_group"] = -1

            idx_from = 0
            for label in range(4):

                idx_to = idx_from + int(cv_truncated_target_ditribution[fold][label] * df.shape[0])

                df["predicted_group"].iloc[idx_from:idx_to] = label

                idx_from = idx_to

            df.loc[df["predicted_group"] < 0, "predicted_group"] = 3
            
            scores.append(qwk(df.accuracy_group, df.predicted_group))
            
        FOLDED_SCORES.append(scores)
        FOLDED_MODELS.append(clf)
        
    scores = list(itertools.chain.from_iterable(FOLDED_SCORES))
    score = np.mean(scores) - np.std(scores)
        
    ALL_CHECKED_CONFIGS.append((
        score,
        (num_leaves, learning_rate),
        [(np.mean(x), np.std(x)) for x in FOLDED_SCORES],
        FOLDED_MODELS,
        FOLDED_SCORES
    ))
    
    return score

Tuning LGB parameters trying to maximize overall mean(qwk) - std(qwk)

In [ ]:
bounds_LGB = {
    'learning_rate': (0.0005, 0.05),
    'num_leaves': (5, 60)
}

LGB_BO = BayesianOptimization(check_cv_config, bounds_LGB, random_state=42)

LGB_BO.probe(
    params={
        'learning_rate': 0.03673,
        'num_leaves': 32
    },
    lazy=True
)

LGB_BO.probe(
    params={
        'learning_rate': 0.0005,
        'num_leaves': 40.53
    },
    lazy=True
)

LGB_BO.probe(
    params={
        'learning_rate': 0.008223,
        'num_leaves': 13.58
    },
    lazy=True
)

LGB_BO.probe(
    params={
        'learning_rate': 0.04857,
        'num_leaves': 20.32
    },
    lazy=True
)

try:
    LGB_BO.maximize(init_points=3, n_iter=25, acq='ucb', xi=0.0, alpha=1e-6)
except Exception as e:
    print("Stop - ", e)

We select multiple trained models, optimal by MEAN - k * STD metric for different k for each fold.

In [ ]:
FINAL_MODELS = []

for fold in range(4):
    print("FOLD", fold+1)
    for sigmas in [0, 1, 2]:
        ALL_CHECKED_CONFIGS.sort(key = lambda x : -(x[2][fold][0] - sigmas * x[2][fold][1]))
        print(ALL_CHECKED_CONFIGS[0][2][fold][0], "+-", ALL_CHECKED_CONFIGS[0][2][fold][1])
        FINAL_MODELS.append(ALL_CHECKED_CONFIGS[0][3][fold])
    print()

For test set, we calculate rank for each point by each model.

Finally, all points sotrted py their average rank.

With this tech we can not bother abouth thresholds fitting and over-fitting, about mixing models with different optimal thresholds and so on.

Once again, it can be heavily punished by completly different private test set. But really, all possible evaluation based on fact that test set has same distribute as train. It is also kind of "not fare" as we can't score one point, only whole test dataset.

On the other hand, is is making our life so mush easier, completly removing threshold-selection layer.

In [ ]:
for i, model in enumerate(FINAL_MODELS):
    pred = model.predict(Xtest)
    submission["pred_%d" % i] = pred
    
for i, model in enumerate(FINAL_MODELS):
    submission.sort_values("pred_%d" % i, inplace = True)
    submission["total_rank"] += np.array(range(submission.shape[0]))
    
submission.sort_values("total_rank", inplace=True)
submission["accuracy_group"] = -1

idx_from = 0
for label in range(4):

    idx_to = idx_from + int(hist[label] * submission.shape[0])

    submission["accuracy_group"].iloc[idx_from:idx_to] = label

    idx_from = idx_to

submission.loc[submission["accuracy_group"] < 0, "accuracy_group"] = 3

submission.to_csv('full_submission.tsv', index=False, sep = '\t')

Writing final submission file

In [ ]:
submission = submission[["installation_id", "accuracy_group"]]

submission.to_csv('submission.csv', index=False)

submission.accuracy_group.value_counts()